# Instructions for users

### The zip file must have a single folder data which should have folders. Each folder represents a class name in the folders only images pertaining to that class must be contained


In [1]:
import zipfile

zip_file = "data.zip"
extract_to = ""

with zipfile.ZipFile(zip_file, "r") as zip_ref:
    zip_ref.extractall(extract_to)

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import layers, models, callbacks
from keras.api.preprocessing import image, image_dataset_from_directory

2024-07-05 21:48:25.403446: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-05 21:48:25.531883: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-05 21:48:25.531916: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-05 21:48:25.566307: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-05 21:48:25.591312: I tensorflow/core/platform/cpu_feature_gua

In [3]:
data_dir = "data"
batch_size = 1
img_height = 120
img_width = 120

train_ds = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)

val_ds = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)


AUTOTUNE = tf.data.experimental.AUTOTUNE

class_names = train_ds.class_names
print(f"class_names = {class_names}")

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

num_classes = len(class_names)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


2024-07-05 21:48:28.052523: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-05 21:48:28.256796: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-05 21:48:28.256982: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
class_names = ['cat', 'dog']


In [5]:
tf.random.set_seed(123)
model = models.Sequential(
    [
        layers.Rescaling(1.0 / 255, input_shape=[img_height, img_width, 3]),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(32, (3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(num_classes),
    ]
)

lr = 0.001
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

early_stopping = callbacks.EarlyStopping(
    monitor="val_accuracy",
    patience=5,
    restore_best_weights=True,
)

In [6]:
epochs = 20
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=[early_stopping],
)

Epoch 1/20


I0000 00:00:1720196327.905611   15662 service.cc:145] XLA service 0x7f07bc003990 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1720196327.905654   15662 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-07-05 21:48:47.950100: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-05 21:48:48.051104: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


   84/20000 ━━━━━━━━━━━━━━━━━━━━ 36s 2ms/step - accuracy: 0.4500 - loss: 0.8913  

I0000 00:00:1720196329.525819   15662 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


20000/20000 ━━━━━━━━━━━━━━━━━━━━ 55s 3ms/step - accuracy: 0.5258 - loss: 0.6995 - val_accuracy: 0.5644 - val_loss: 0.6812
Epoch 2/20
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 30s 2ms/step - accuracy: 0.5736 - loss: 0.6768 - val_accuracy: 0.6474 - val_loss: 0.6236
Epoch 3/20
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 30s 2ms/step - accuracy: 0.6465 - loss: 0.6234 - val_accuracy: 0.6846 - val_loss: 0.5909
Epoch 4/20
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 30s 1ms/step - accuracy: 0.7025 - loss: 0.5730 - val_accuracy: 0.7150 - val_loss: 0.5634
Epoch 5/20
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 30s 2ms/step - accuracy: 0.7217 - loss: 0.5468 - val_accuracy: 0.7150 - val_loss: 0.5680
Epoch 6/20
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 30s 1ms/step - accuracy: 0.7334 - loss: 0.5257 - val_accuracy: 0.7336 - val_loss: 0.5454
Epoch 7/20
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 30s 2ms/step - accuracy: 0.7541 - loss: 0.5009 - val_accuracy: 0.7310 - val_loss: 0.5556
Epoch 8/20
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 30s 2ms/step - accuracy: 0.7714 - lo

In [7]:
val_loss, val_acc = model.evaluate(val_ds, verbose=2)
print(f"Validation accuracy: {val_acc}")

# Save the model
model.save("model.h5")

5000/5000 - 3s - 683us/step - accuracy: 0.7466 - loss: 0.5603


Validation accuracy: 0.7465999722480774


In [8]:
model.save("model.h5")

In [9]:
def predict_image(image_path):
    img = image.load_img(image_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    print(
        "This image most likely belongs to {} with a {:.2f} percent confidence.".format(
            class_names[tf.argmax(score)], 100 * tf.reduce_max(score)
        )
    )

In [11]:
predict_image("data/cat/cat.1.jpg")
predict_image("data/dog/dog.1.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


This image most likely belongs to cat with a 99.86 percent confidence.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
This image most likely belongs to dog with a 93.06 percent confidence.
